In [1]:
from nltk.stem.porter import PorterStemmer
import string
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from pickle import dump
nltk.download('stopwords')
from string import punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
ps = PorterStemmer()
stop_words = set(stopwords.words('indonesian'))

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset = [j for j in open('/content/drive/MyDrive/Bahan Tugas Akhir/CNN/dataset/tagged_data.txt',encoding="utf8").read().split("\n") if j !='']
print(dataset[:11])

['rt @napqilla: no 1, 3 ambisinya menguasai rakyat, no.2 ambisinya melayani rakyat, itu terlihat #debat2pilkadadki, mknya #loguepilihbadja aja;1', 'rt @pandji: nah gue pikir sentimen petahana oke di #debat2pilkadadki semalam. tapi mnrt politicawave malah sebaliknya. geng\\u2026 ;1', 'rt @pandji: urutan pertama best moment #debat2pilkadadki : pak basuki misahin bu sylvie &amp;1', 'rt @pandji: ini artikel yg menjelaskan ternyata di #debat2pilkadadki yg dapet respon positif di socmed adalah #aniessandiokoce https:\\/\\/t.co\\u2026;1', 'rt @mrtampi: agus makin santai.\\nahok makin santun.\\nanies makin esmosi.\\n\\nimho.\\n#debat2pilkadadki;0', 'pilkada dki jangan pilih pki!! berbahaya!! pilih 1 atau 3 saja tahun baru imlek, pkb aj... https:\\/\\/t.co\\/ggych4sgct #debat2pilkadadki #imlek;0', 'pdip sengaja becah belah rakyat warga dki tidak pilih ahok!! wni ang... ;0', 'nama nya juga debat\\nbiasa nya itu https:\\/\\/t.co\\/cjertypdxp;1', 'rt @dadansuwarna: kita ttp bisa bijak memilih, tan

In [5]:
#preprocess per sentence
def clean_doc(doc):
    #split into tokens by white space
    tokens = doc.split()
    #remove punctuation from each token
    table  = str.maketrans('','',punctuation)
    tokens = [w.translate(table) for w in tokens]
    #remove remaining tokens that not alphabetic
    #tokens = [word for word in tokens if word.isalpha()]
    #filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    #filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = ' '.join(tokens)
    return tokens

In [6]:
dataset_cln = []
positive_dt =[]
negative_dt =[]

for line in dataset:
    line = line.split(";")
    if line[1] == "1":
        positive_dt.append(clean_doc(line[0]))
    else:
        negative_dt.append(clean_doc(line[0]))
    
print(len(positive_dt))
print(len(negative_dt))

757
749


In [7]:
positive_dt[:11]

['rt napqilla ambisinya menguasai rakyat no2 ambisinya melayani rakyat itu terlihat debat2pilkadadki mknya loguepilihbadja aja',
 'rt pandji nah gue pikir sentimen petahana oke di debat2pilkadadki semalam tapi mnrt politicawave malah sebaliknya gengu2026',
 'rt pandji urutan pertama best moment debat2pilkadadki pak basuki misahin bu sylvie amp',
 'rt pandji ini artikel yg menjelaskan ternyata di debat2pilkadadki yg dapet respon positif di socmed adalah aniessandiokoce httpstcou2026',
 'nama nya juga debatnbiasa nya itu httpstcocjertypdxp',
 'rt dadansuwarna kita ttp bisa bijak memilih tanpa harus berteriak tanpa harus mengumpat debat2pilkadadki debatcagubdki',
 'rt aniesbaswedan ibu terima kasih atas doa dan kumpulan pelajaran yang telah membangun pemikiran karakter dan semangat untuk berjuu2026',
 'rt jokoanwar tiap ahok djarot ngomong nambah ilmu ya kita debat2pilkadadki httpstcoxwwrnalpbu',
 'rt jessyastrid makasih yosimokalu jangan kasih kendor untuk 1putaran 2periode debat2pilkada

In [8]:
trainX = negative_dt[:500] + positive_dt[:500]
#labeling dataset, for first length of negative post, label with 0 for next length of positive post, label with 1
trainy = [0 for _ in range(500)] + [1 for _ in range(500)]


In [9]:
# trainX[:11]

In [10]:
def save_dataset(dataset, filename):
    dump(dataset, open(filename,'wb'))
    print('saved %s'% filename)

save_dataset([trainX,trainy], 'train.pkl')

saved train.pkl


In [25]:
testX = negative_dt[501:] + positive_dt[501:]
testY = [0 for _ in range(len(negative_dt[501:]))] + [1 for _ in range(len(positive_dt[501:]))]
save_dataset([testX,testY],'test.pkl')

saved test.pkl


In [32]:
# testX

In [12]:
import pickle

def load_dataset(filename):
    return pickle.load(open(filename,'rb'))

trainLines, trainLabels = load_dataset('train.pkl')

In [13]:
len(trainLines)

1000

In [14]:
# trainLabels

In [15]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate


In [16]:
#load the cleaned dataset
def load_dataset(filename):
    return pickle.load(open(filename,'rb'))

#fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def max_length(lines):
    return max([len(s.split()) for s in lines])

def encode_text(tokenizer, lines, length):
    #integer encoded
    encoded = tokenizer.texts_to_sequences(lines)
    #pad encoded sequences
    padded = pad_sequences(encoded, maxlen = length, padding='post')
    return padded

def define_model(length, vocab_size):
    #channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    #channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size = 6, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = Dropout(0.5)(conv2)
    flat2 = Flatten()(pool2)
    
    #channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    #merge
    merged = concatenate([flat1, flat2, flat3])
    
    #interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    
    #compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #summarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [17]:
trainLines, trainLabels = load_dataset('train.pkl')
#create tokenizer
tokenizer = create_tokenizer(trainLines)
#calculate max document length
length = max_length(trainLines)
#calculate vocabulary size
vocab_size = len(tokenizer.word_index)+1
print('Max document length : %d' % length)
print('vocabulary size : %d' % vocab_size)

#encode data
trainX = encode_text(tokenizer, trainLines, length)
print(trainX.shape)

#define model
model = define_model(length, vocab_size)
#fit model
model.fit([trainX, trainX, trainX], array(trainLabels), epochs=10, batch_size = 16)
#save the model
model.save('model.h5')

Max document length : 39
vocabulary size : 3427
(1000, 39)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 39)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 39)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 39, 100)      342700      ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 39)]         0           []                               
                                   